In [ ]:
#!pip install spotipy
#!pip install mutagen
#!pip install yt_dlp
#!pip install youtube_search
#!pip install -v pandas==0.23.1
#!pip install numpy
#!pip uninstall youtube_search
#!pip install git+https://github.com/LeeMinNguyeen/youtube_search_35python.git

In [ ]:
import youtube_search
import YanAPI as YanAPI
from mutagen.mp3 import MP3
import time

ip_addr = "192.168.1.12" # please change to your yanshee robot IP
YanAPI.yan_api_init(ip_addr)

In [ ]:
import yt_dlp
from youtube_search import YoutubeSearch

def download_music(search_str):
    YTSearchResult = YoutubeSearch(search_str, max_results=1).to_dict()

    URLS = "https://www.youtube.com{link}".format(link = YTSearchResult[0]['url_suffix'])

    ydl_opts = {
        'format': 'mp3/bestaudio/best',
        'ffmpeg_location': './ffmpeg-git-fullbuild/bin',
        'outtmpl': './Music/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download((URLS,))
    
    return ydl_opts['outtmpl']

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def find_music(search_str):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id="b89acd4356eb4d0ca884d29d568daa20",
        client_secret="ec186bc7b2fc4aeeb4afd41afda26cad"
        )
    )

    results = sp.search(search_str, limit=1)
    
    result_details = dict(
        SongName = results['tracks']['items'][0]['name'],
        MainArtist = results['tracks']['items'][0]['artists'][0]['name'],
        #FeaturedArtist = AssistArtist,
        #URI = results['tracks']['items'][0]['uri']
    )

    #result = pd.DataFrame(search_result)

    #result.to_csv('./Result.csv')

    search_result = str(result_details['SongName'] + ' ' + result_details['MainArtist'])

    return search_result

In [ ]:
def get_music():
    while True:
        YanAPI.start_voice_tts("What do want to listen to",False)
        time.sleep(2) #sleep 2 seconds to ensure the voice will not be interapt by the code below
        listen_res = YanAPI.sync_do_voice_asr_value() # listen again to get the name.
        print(listen_res)
        if len(listen_res["question"]) > 0:# user said something
            YanAPI.start_voice_tts("Please wait while I find the song",False)
            result = find_music(str(listen_res['question']))
            YanAPI.start_voice_tts("Is this what you are looking for: "+result+"",False)
            #if YanAPI.sync_do_voice_asr_value()['question'] == 'no':
                #continue
            #else:
            YanAPI.start_voice_tts("Please wait while I download the song",False)
            break
        else:
            YanAPI.start_voice_tts("I cannot hear you, try again",False)
    return listen_res['question']

def play_song():
    title = download_music(get_music())
    YanAPI.start_play_music('./Music/{title}'.format(title=title))
    time.sleep(5)
    # time.sleep(MP3(f'./Music/{title}').info.length)
    YanAPI.stop_play_music()

In [ ]:
play_song()